# Countries circulation

<font size="4">which journal(s) initiate(s) chains of circulations outside their country of publication</font>

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import sparql_dataframe
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt
import folium
import VizKG.visualize as vkg
import plotly.express as px

In [2]:
endpoint = "http://localhost:3030/VisualContagions/sparql"

In [3]:
q ="""

PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>
PREFIX frbroo: <http://iflastandards.info/ns/fr/frbr/frbroo/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX aat: <http://vocab.getty.edu/aat/>


SELECT DISTINCT ?cluster ?identifier ?AGJournal ?placeAGLabel ?notAGJournal ?placeNOTAGLabel WHERE {

  
	VALUES ?avantgarde { <https://lod.unige.ch/vc/type/988EE2E2-172C-3104-8DAA-B4D629E814AE> } .
	

		?cluster crm:P148_has_component ?image ;
           	crm:P1_is_identified_by ?identifier .
		?image crm:P128i_is_carried_by ?issue . 
		?issue crm:P2_has_type ?avantgarde;
                crm:P108i_was_produced_by ?productionAG ;
				rdfs:label ?AGJournal.  

        ?productionAG crm:P7_took_place_at ?placeAG .
        ?placeAG crm:P2_has_type aat:300387506 ;
                rdfs:label ?placeAGLabel .
	
		?cluster crm:P148_has_component ?image1 . 
		?image1 crm:P128i_is_carried_by ?issue1 . 
		?issue1 crm:P2_has_type ?type ;
                crm:P108i_was_produced_by ?productionNOTAG ;
				rdfs:label ?notAGJournal .
		?type rdfs:label ?JournalTypeLabel .
        
        ?productionNOTAG crm:P7_took_place_at ?placeNOTAG .
        ?placeNOTAG crm:P2_has_type aat:300387506 ;
                rdfs:label ?placeNOTAGLabel .

		FILTER (contains(str(?type), "https://lod.unige.ch/vc/type/"))
		FILTER (?type != ?avantgarde)
        FILTER (?placeAGLabel != ?placeNOTAGLabel)
        

# Trashy cluster not useful for the analysis

	FILTER NOT EXISTS {
                          VALUES ?trash { <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000040066b73cf2a0b0742a0aceecc4dd1724b52dc7afbdc4b48432ee3f0336124798aa> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001200bed2a9ade1689744b661b58c1d93bcd442cbeafcf07b6460d0df1b33d65f5b86> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000022005d01a84e38421213b8816fd78e4aef0890445f65f81de2ee1459980a6335942a> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001600fe63944181de71b69fc3900867dc30611de913a214c5d42ac2e7ce3dd7cff99f> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000012009d66187e47b439dd2c2e6668dd9945209f40685c7039c56c563a4aa9bbfa9718> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000d0069b0d3bd802ae948c1f53b96391afeb46fcd505e999cb56cd37406508e4ef418> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000b002057ee6db2a8ea72c04e36379e7b035a4d2911a1155b5652230551866825ab90> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000b0003d9d58d012451307b2ef5fbd75af31eec0c5f1afcdefb10b4ab8fbc38d15615> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000a00609f4e0b53f5b2ca7afaea13c7a3ba187b05ae3bdd1108598e5226a7aaf18ded> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000008005b033c554cfd743ed238cd05b35f46fe7de56720622971b00aecaf985896fe4a> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00000600d4e85eaf39fb676e999b8d2466a4455d48d70791ff2d4dd1cc4653c248264df3> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00002a00a60bf43cf1fb052165221ee1727e977b2ecf99f820cbbd426d12c6aa6a37d9e4> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000310098549f748176d2f4efedf76773dca2aafeaf8db5f746daf37a50dd3d0819dbb3> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0003d500d8362a66c04fe8273b00d9470929fd918a6dc6c4395f060e0c5e270f12f54f5c> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0003ff00a6da95a6b61b8c47220d9839b7b4b6afe26daad02ee0c0c291c40ad1bb0bec90> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000c9000f961bd877cdc346bd51b76a74b698924ae6960e3bbfa0986e7aea7da730930c> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0000d000ecf8a0329d2530d3eb601e077c0e5c298db100b8316c0af9232e41e8a7e8e4ef> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000403008f37a6852f6339eefe0e74225b20553b5d641b32015b15d40ca4d216a540ae2f> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00002400d7b4ea535865b256e1c13f0d6505182ead425183c8f919bfa3309aab1dae43dc> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001f00c18ad27c34a66610ece997934ce4bc1aa5fbcdad562e58a52ab86792352bfb9c> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00002b000918272590bc01401e6367c6448a222445fe13cabec0a2a38a7b38e598cea622> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000057008d59ea30b6175a9c5c32ebf3caa2c27cce0d569fd3b2a899ebf0107086c3a197> <https://visualcontagions.unige.ch/explore/duplicates/clusters/000028007bf1c23529418f9fba6de2955caa8fa4e1572632151c41a9f1b20c1c5a45a1c5> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00005a009d47a1932fcbbe7ff04ac3017fa6d167f06fe3287477d6c1cedae50451cd67dc> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001f0087307813ef1d16a6def5d96b8b6550a707da4fd6702aa64d8425b29e6826f049> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001c0000968f38146238bab2f376c40b0fca62e734f151a85e2ecce4981e0050eb57a0> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001d00ffc088f4f254a86d70e0a3495bbf5f288b1373e96c6875d487cd1b14076e2702> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001d000d01d183fa765191f6028c4b69a7e1c9c9ed1d9a3ae14bb9a4731b5172f4b337> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00001c002016002f400721bfa0a32f16e887e64cc4e092cd626a7fb7c426545e0757e7f1> <https://visualcontagions.unige.ch/explore/duplicates/clusters/0002910088db3f58b1c573c281f266c695c21d421c9337abca57d4a51440e13a2f9c5893> <https://visualcontagions.unige.ch/explore/duplicates/clusters/00028200ce1d42e579bb46d53b1b038b5f23bf88a8057f5d5cb9806d9c87dee28477ecc4>
                                       } .
    ?cluster crm:P1_is_identified_by  ?trash . } 

}



"""

In [4]:
df = sparql_dataframe.get(endpoint, q)

In [5]:
df

,cluster,identifier,AGJournal,placeAGLabel,notAGJournal,placeNOTAGLabel
0,https://lod.unige.ch/vc/cluster/00000400736029...,https://visualcontagions.unige.ch/explore/dupl...,"Mirador : setmanari de literatura, art i política",Spain,Archiv für Buchgewerbe und Gebrauchsgraphik,Germany
1,https://lod.unige.ch/vc/cluster/00000400736029...,https://visualcontagions.unige.ch/explore/dupl...,"Mirador : setmanari de literatura, art i política",Spain,Pan,Germany
2,https://lod.unige.ch/vc/cluster/000002002c3c55...,https://visualcontagions.unige.ch/explore/dupl...,"Mirador : setmanari de literatura, art i política",Spain,Commercial Art - Art and Industry,United Kingdom
3,https://lod.unige.ch/vc/cluster/00004e00af3886...,https://visualcontagions.unige.ch/explore/dupl...,"Mirador : setmanari de literatura, art i política",Spain,Das Buch für alle,Germany
4,https://lod.unige.ch/vc/cluster/00004e00af3886...,https://visualcontagions.unige.ch/explore/dupl...,"Mirador : setmanari de literatura, art i política",Spain,Berliner illustrierte Zeitung,Germany
...,...,...,...,...,...,...
5039,https://lod.unige.ch/vc/cluster/000002002a2d3c...,https://visualcontagions.unige.ch/explore/dupl...,Dokumentum,Hungary,Der Cicerone,Germany
5040,https://lod.unige.ch/vc/cluster/00000a0065d77d...,https://visualcontagions.unige.ch/explore/dupl...,Dokumentum,Hungary,Freie Kunst und Literatur : liberté pour l'art...,France
5041,https://lod.unige.ch/vc/cluster/00000a0065d77d...,https://visualcontagions.unige.ch/explore/dupl...,Dokumentum,Hungary,Le Cap : critique art philosophie : bulletin m...,France
5042,https://lod.unige.ch/vc/cluster/0000030085b306...,https://visualcontagions.unige.ch/explore/dupl...,Dokumentum,Hungary,Le rire : journal humoristique,France


In [6]:
df.sort_values(by=['identifier'], ascending=False, inplace=True)

In [7]:
df['count_cluster'] = df.groupby(['identifier'])['cluster'].transform('count')

In [14]:
df.to_csv('countries.csv', index=False)

In [8]:
df['count_country'] = df.groupby(['placeNOTAGLabel'])['notAGJournal'].transform('count')

In [9]:
df['count_notAGJournal'] = df.groupby(['notAGJournal'])['placeNOTAGLabel'].transform('count')

In [17]:
df

,cluster,identifier,AGJournal,placeAGLabel,notAGJournal,placeNOTAGLabel,count_cluster,count_country,count_notAGJournal
2420,https://lod.unige.ch/vc/cluster/0003c40026cf13...,https://visualcontagions.unige.ch/explore/dupl...,Con V de Vian,Argentina,La Marianne (Paris. 1906),France,17,1192,5
2418,https://lod.unige.ch/vc/cluster/0003c40026cf13...,https://visualcontagions.unige.ch/explore/dupl...,Con V de Vian,Argentina,La Jeunesse illustrée,France,17,1192,3
2429,https://lod.unige.ch/vc/cluster/0003c40026cf13...,https://visualcontagions.unige.ch/explore/dupl...,Con V de Vian,Argentina,Weltkunst,Germany,17,495,10
2428,https://lod.unige.ch/vc/cluster/0003c40026cf13...,https://visualcontagions.unige.ch/explore/dupl...,Con V de Vian,Argentina,Black Cat; A Monthly Magazine of Original Shor...,United States of America,17,260,7
2427,https://lod.unige.ch/vc/cluster/0003c40026cf13...,https://visualcontagions.unige.ch/explore/dupl...,Con V de Vian,Argentina,Modern Publicity,United Kingdom,17,161,6
...,...,...,...,...,...,...,...,...,...
183,https://lod.unige.ch/vc/cluster/00000200036bbe...,https://visualcontagions.unige.ch/explore/dupl...,Praesens,Poland,Das neue Frankfurt,Germany,1,495,31
2952,https://lod.unige.ch/vc/cluster/0000020002bcc0...,https://visualcontagions.unige.ch/explore/dupl...,Zivot,Czech Republic,Cahiers d'art (Paris),France,1,1192,74
2964,https://lod.unige.ch/vc/cluster/0000020000dbab...,https://visualcontagions.unige.ch/explore/dupl...,Horizont,Czech Republic,Deutsche Kunst und Dekoration,Germany,1,495,36
2038,https://lod.unige.ch/vc/cluster/000002000089cc...,https://visualcontagions.unige.ch/explore/dupl...,Minotaure,France,Der Cicerone,Germany,1,495,64


In [11]:
df_viz = df[df['count_country'] >= 100]

fig = px.sunburst(df_viz, path=['placeAGLabel', 'placeNOTAGLabel'], values='count_notAGJournal')
fig.update_layout(uniformtext=dict(minsize=10, mode='hide'))
#fig.show()
fig.write_html('country_sunburst.html', auto_open=True)